# 注意力机制

## 点积

缩放点积注意力是一种注意力机制，其中点积按 $\sqrt{d_k}$比例缩小 .Formally we have a query $Q$, a key $K$ and a value $V$ and calculate the attention as:

$$
\text{Attention}(Q,K,V)=\text{softmax}\biggl(\frac{QK^T}{\sqrt{d_k}}\biggr)V
$$

我们假设 $q$ and $k$ are $d_k$-dimensional vectors whose components are independent random variables with mean 0 and variance 1, then their dot product, $q\cdot k=\sum_{i=1}^{dk}u_iv_i$ has mean 0 and variance
 $d_k$.Since we would prefer these values to have variance 1, we divide by $\sqrt{d_k}$

 ```{figure} ../images/attention/scaled-dot-product-attention.png
:width: 200px
:align: center
:name: my-fig-ref

My figure title.
```


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ScaledDotProductAttention(nn.Module):
    ''' Scaled Dot-Product Attention '''

    def __init__(self, temperature, attn_dropout=0.1):
        super().__init__()
        self.temperature = temperature
        self.dropout = nn.Dropout(attn_dropout)

    def forward(self, q, k, v, mask=None):

        attn = torch.matmul(q / self.temperature, k.transpose(2, 3))

        if mask is not None:
            attn = attn.masked_fill(mask == 0, -1e9)

        attn = self.dropout(F.softmax(attn, dim=-1))
        output = torch.matmul(attn, v)

        return output, attn

## self attention

Self Attention在2017年Google机器翻译团队发表的《Attention is All You Need》中被提出，它完全抛弃了RNN和CNN等网络结构，而仅采用新提出的Self Attention机制来处理机器翻译任务，并且取得了很好的效果。

在Encoder-Decoder框架下，广义的attention机制中的输入Source和输出Target内容是不一样的，以英-中机器翻译为例，Source是英文句子，Target是对应的翻译出的中文句子，Attention机制发生在Target的元素和Source中的所有元素之间。此时Query来自Target， Key和Value来自Source。

而Self Attention顾名思义，指不是Target和Source之间做Attend，而是Source内部元素之间或者Target内部元素之间发生的Attention机制，也可以理解为Target=Source这种特殊情况下的注意力计算机制。此时Query、Key和Value都来自Target或Source。

Attention和Self Attention的本质上都是为序列中每个元素都分配一个权重系数，这也可以理解为软寻址。如果序列中每一个元素都以(K,V)形式存储，那么attention则通过计算Q和K的相似度来完成寻址。Q和K计算出来的相似度反映了取出来的V值的重要程度，即权重，然后加权求和就得到了attention值（实际上就是一种learnable weighted sum）。

首先，向量点积可以表征向量之间的相关程度，点积越大越相关。softmax(QKT)
就是自身与自身求了相关性，得到一个所有数值为0-1的mask矩阵，即weights；是query和key的输入的第一维大小，也就是multi-head的每个head上的feature dims，对QKT
做scaling可以避免其值特别大的时输入到softmax之后会得到极小的gradient的情况；同时学到合适的V以保留输入的特征，用weights对V做加权求和得到attention values。

Scaled dot-product attention is an attention mechanism where the dot products are scaled down by $\sqrt{d_k}$ .Formally we have a query $Q$, a key $K$ and a value $V$ and calculate the attention as:

$$
\text{Attention}(Q,K,V)=\text{softmax}\biggl(\frac{QK^T}{\sqrt{d_k}}\biggr)V
$$

If we assume that $q$ and $k$ are $d_k$-dimensional vectors whose components are independent random variables with mean 0 and variance 1, then their dot product, $q\cdot k=\sum_{i=1}^{dk}u_iv_i$ has mean 0 and variance
 $d_k$.Since we would prefer these values to have variance 1, we divide by $\sqrt{d_k}$

 ```{figure} ../images/attention/muti-head-attention.png
:width: 200px
:align: center
:name: my-fig-ref

My figure title.
```

https://paperswithcode.com/method/scaled

In [ ]:
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from transformer.Modules import ScaledDotProductAttention

__author__ = "Yu-Hsiang Huang"

class MultiHeadAttention(nn.Module):
    ''' Multi-Head Attention module '''

    def __init__(self, n_head, d_model, d_k, d_v, dropout=0.1):
        super().__init__()

        self.n_head = n_head
        self.d_k = d_k
        self.d_v = d_v

        self.w_qs = nn.Linear(d_model, n_head * d_k, bias=False)
        self.w_ks = nn.Linear(d_model, n_head * d_k, bias=False)
        self.w_vs = nn.Linear(d_model, n_head * d_v, bias=False)
        self.fc = nn.Linear(n_head * d_v, d_model, bias=False)

        self.attention = ScaledDotProductAttention(temperature=d_k ** 0.5)

        self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(d_model, eps=1e-6)


    def forward(self, q, k, v, mask=None):

        d_k, d_v, n_head = self.d_k, self.d_v, self.n_head
        sz_b, len_q, len_k, len_v = q.size(0), q.size(1), k.size(1), v.size(1)

        residual = q

        # Pass through the pre-attention projection: b x lq x (n*dv)
        # Separate different heads: b x lq x n x dv
        q = self.w_qs(q).view(sz_b, len_q, n_head, d_k)
        k = self.w_ks(k).view(sz_b, len_k, n_head, d_k)
        v = self.w_vs(v).view(sz_b, len_v, n_head, d_v)

        # Transpose for attention dot product: b x n x lq x dv
        q, k, v = q.transpose(1, 2), k.transpose(1, 2), v.transpose(1, 2)

        if mask is not None:
            mask = mask.unsqueeze(1)   # For head axis broadcasting.

        q, attn = self.attention(q, k, v, mask=mask)

        # Transpose to move the head dimension back: b x lq x n x dv
        # Combine the last two dimensions to concatenate all the heads together: b x lq x (n*dv)
        q = q.transpose(1, 2).contiguous().view(sz_b, len_q, -1)
        q = self.dropout(self.fc(q))
        q += residual

        q = self.layer_norm(q)

        return q, attn
